Link dos datasets

Samba: https://www.kaggle.com/datasets/clovesgtx/brazilian-music-samba-lyrics

Gospel: https://raw.githubusercontent.com/damarals/letras/master/inst/csv/letras.csv

Axe: https://www.kaggle.com/datasets/jorgefjr/brazilian-songs-lyrics

Funk: https://www.kaggle.com/datasets/sleshes/funk-carioca-dataset

Bossa Nova: https://www.kaggle.com/datasets/mcarujo/bossa-nova-lyrics

Rock,Sertanejo, Samba2 = https://github.com/gabriellmd/data-frames-nlp

In [2]:
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

In [84]:
bossa = pd.read_csv('datasets/bossa_nova_songs_dataset.csv')
samba = pd.read_csv('datasets/samba_dataset.csv',sep='|')
axe = pd.read_csv('datasets/letras_mus_br_axe.csv')
funk = pd.read_csv('datasets/Dataset_funk_carioca.csv')
gospel = pd.read_csv('datasets/gospel.csv') 

In [85]:
gospel_ajustado = pd.DataFrame(gospel['letra'])  #OK 55000
gospel_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
gospel_ajustado.drop(gospel_ajustado.tail(51498).index,inplace = True) #3000
samba_ajustado = pd.DataFrame(samba['letra'])  #OK 3000
samba_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
samba_ajustado.drop(samba_ajustado.tail(118).index,inplace = True) #3000
axe_ajustado = pd.DataFrame(axe['letras'])  #OK1000
axe_ajustado.rename(columns={'letras': 'lyric'}, inplace = True) #OK
funk_ajustado = pd.DataFrame(funk['letra'])  #OK 1000
funk_ajustado.rename(columns={'letra': 'lyric'}, inplace = True) #OK
bossa_ajustado = pd.DataFrame(bossa['song_lyrics'])  #OK 6000
bossa_ajustado.rename(columns={'song_lyrics': 'lyric'}, inplace = True) #OK
bossa_ajustado.drop(bossa_ajustado.tail(3106).index,inplace = True) #3000

In [86]:
bossa_ajustado['gen'] = 'Bossa Nova'
funk_ajustado['gen'] = 'Funk'
gospel_ajustado['gen'] = 'Gospel'
axe_ajustado['gen'] = 'Axe'
samba_ajustado['gen'] = 'Samba'

In [87]:
datasets = [bossa_ajustado,funk_ajustado,bossa_ajustado,axe_ajustado,samba_ajustado]

In [88]:
lyrics = pd.concat(datasets)
lyrics.reset_index(drop=True,inplace=True)
lyrics

,lyric,gen
0,"De tudo, ao meu amor serei atento antes E co...",Bossa Nova
1,Era uma casa Muito engraçada Não tinha teto ...,Bossa Nova
2,E por falar em saudade Onde anda você Onde a...,Bossa Nova
3,Filhos... Filhos? Melhor não tê-los! Mas se ...,Bossa Nova
4,É melhor ser alegre que ser triste Alegria é...,Bossa Nova
...,...,...
10995,( Não tenho lar Não tenho amor Pra que vou me ...,Samba
10996,o Senhor é a minha forçao meu escudo...minha ...,Samba
10997,Sorte é sorte o malandro também tem seu dia d...,Samba
10998,"A gente briga sem querer Chora ""prá"" voltar F...",Samba


In [89]:
def processamento_texto(txt):
    # remover quebras de linha
    txt = txt.replace('\n',' ')
    # remover símbolos de pontuação, resultando em um array de caracteres
    txt = [char for char in txt if char not in string.punctuation]
    # depois, juntar os caracteres em palavras novamente e separá-los em uma lista de tokens
    txt = ''.join(txt).split()
    # por fim, remover as stopwords da lista
    #txt = [word for word in txt if word.lower() not in stopwords.words('portuguese')]
    
    return txt

In [90]:
alg ={'SVM':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
},
     'RF':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
},
     'NB':{
    'fit_time':np.array([]),
    'score_time':np.array([]),
    'test_accuracy':np.array([]),
    'test_f1_score':np.array([]),
} }

In [91]:
for count in range(5):
    #aleatorio
    lyric_train,lyric_test,gen_train,gen_test = train_test_split(lyrics['lyric'],lyrics['gen'],test_size=0.3)
    #dados SVC
    pipeline_svc = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('svc',SVC())
    ])
    #dados NB
    pipeline_nb = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('nb',MultinomialNB())
    ])
    #dadps RF
    pipeline_rf = Pipeline([
    ('bow',CountVectorizer(analyzer=processamento_texto,encoding ='unicode')),
    ('tfidf',TfidfTransformer()),
    ('rf',RandomForestClassifier())
    ])
    
    #Etapa de Fit
    pipeline_svc.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    pipeline_nb.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    pipeline_rf.fit(lyric_train.values.astype('U'),gen_train.values.astype('U'))
    
    #Etapa 10-fold cross validation
    scoring = {'accuracy' : 'accuracy', 'f1_score' : 'f1_micro'}
    scores_svc = cross_validate(pipeline_svc,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    scores_nb = cross_validate(pipeline_nb,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    scores_rf = cross_validate(pipeline_rf,lyrics['lyric'].values.astype('U'),lyrics['gen'].values.astype('U'),cv=10,scoring=scoring)
    
    #Aninhamento de médias
    #SVM
    alg['SVM']['fit_time'] = np.append(alg['SVM']['fit_time'], scores_svm['fit_time'].mean())
    alg['SVM']['score_time'] = np.append(alg['SVM']['score_time'], scores_svm['score_time'].mean())
    alg['SVM']['test_accuracy'] = np.append(alg['SVM']['test_accuracy'], scores_svm['test_accuracy'].mean())
    alg['SVM']['test_f1_score'] = np.append(alg['SVM']['test_f1_score'], scores_svm['test_f1_score'].mean())
    #NB
    alg['NB']['fit_time'] = np.append(alg['NB']['fit_time'], scores_nb['fit_time'].mean())
    alg['NB']['score_time'] = np.append(alg['NB']['score_time'], scores_nb['score_time'].mean())
    alg['NB']['test_accuracy'] = np.append(alg['NB']['test_accuracy'], scores_nb['test_accuracy'].mean())
    alg['NB']['test_f1_score'] = np.append(alg['NB']['test_f1_score'], scores_nb['test_f1_score'].mean())
    #RF
    alg['RF']['fit_time'] = np.append(alg['RF']['fit_time'], scores_rf['fit_time'].mean())
    alg['RF']['score_time'] = np.append(alg['RF']['score_time'], scores_rf['score_time'].mean())
    alg['RF']['test_accuracy'] = np.append(alg['RF']['test_accuracy'], scores_rf['test_accuracy'].mean())
    alg['RF']['test_f1_score'] = np.append(alg['RF']['test_f1_score'], scores_rf['test_f1_score'].mean())
    

In [93]:
alg['RF']['fit_time']

array([85.2930145 , 86.02140307, 84.9799845 , 85.01941352, 86.04448705])

In [94]:
alg['RF']['score_time']

array([0.27017698, 0.26408484, 0.25601187, 0.29369226, 0.28299279])

NameError: name 'alg' is not defined